In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import src.geo_functions as gf
from bson.json_util import dumps
import json
import pandas as pd

# Importing the madrid functions:
import src.geo_functions as gf

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("geospatial")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'geospatial')

In [3]:
lisbon = db.get_collection("lisbon")
london = db.get_collection("london")
madrid = db.get_collection("madrid")

In [ ]:
# Creamos el  index 2dsphere desde pymongo

db.lisbon.create_index([("location", GEOSPHERE)])
db.london.create_index([("location", GEOSPHERE)])
db.madrid.create_index([("location", GEOSPHERE)])


In [4]:
diccionario = {"Pub": "Bar",
            "Metro": 'Train Station',
            "Café": "Bar",
            "Bar": "Bar",
            "Comedy Club": "Bar",
            "Steakhouse": "Bar",
            "Hotel Bar": "Bar",
            "Cocktail":"Bar",
            "Lounge": "Bar",
            "Nightclub": "Bar",
            "Restaurant": "Bar",
            "Sports Bar": "Bar", 
            "Bistro": "Bar",
            "Wine Bar": "Bar",
            "Pizza": "Bar",  
            'Bus Stop': 'Train Station',
            "Nightclub": "Bar",
            "Spanish": 'Bar',
            "Winery": "Bar",
            "Cocktail": "Bar",
            "Tapas": "Bar",
            "Coffee Shop": "Starbucks", 
            "Japanese": "Bar",
            "Seafood": "Bar",  
            "Brewery":"Bar",
            "Diner": "Bar",
            "Breakfast": "Bar",
            "Chinese": "Bar",
            'New American': 'Bar',
            "Cafeteria": "Bar",
            "Salon / Barbershop": "Bar",
            "Roof Deck": "Bar",
            "Sushi": "Bar", 
            "Snacks": "Bar",  
            "Karaoke":"Bar",
            "Corporate Cafeteria": "Bar",
            "Ice Cream": "Bar",
            'Portuguese': 'Bar',
            'Train Station': 'Train Station'}

# LONDON 

In [9]:
london_ = gf.geocode("St Paul´s Cathedral, London")

In [10]:
london_

{'type': 'Point', 'coordinates': ['51.51421', '-0.05688']}

In [11]:
query_london = [{"$geoNear" : {'near': [51.51421, -0.05688],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

In [12]:
geoloc_london = london.aggregate(query_london)

In [13]:
london_json = json.loads(dumps(geoloc_london))

In [21]:
london_df = pd.DataFrame(london_json)

In [22]:
gf.drop_columns(london_df)

In [23]:
london_df["city"] = "London"

In [24]:
london_df['shortname'] = london_df['shortname'].map(diccionario)

In [25]:
london_df.shortname.unique()

array(['Train Station', 'Bar', 'Starbucks', nan], dtype=object)

# LISBON

In [28]:
lisbon_ = gf.geocode("Avenida Antonio Augusto de Aguiar, Lisboa")

In [29]:
lisbon_

{'type': 'Point', 'coordinates': ['38.73280', '-9.15221']}

In [30]:
query_lisbon = [{"$geoNear" : {'near': [38.73280, -9.15221],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

In [31]:
geoloc_lisbon = lisbon.aggregate(query_lisbon)

In [32]:
lisbon_json = json.loads(dumps(geoloc_lisbon))

In [33]:
lisbon_df = pd.DataFrame(lisbon_json)

In [34]:
gf.drop_columns(lisbon_df)

In [35]:
lisbon_df["city"] = "Lisbon"

In [36]:
lisbon_df['shortname'] = lisbon_df['shortname'].map(diccionario)

In [37]:
lisbon_df.shortname.unique()

array(['Bar', 'Starbucks', 'Train Station'], dtype=object)

# MADRID

In [38]:
madrid_ = gf.geocode("Paseo de la Castellana, Madrid")

In [39]:
madrid_

{'type': 'Point', 'coordinates': ['40.46879', '-3.68915']}

In [40]:
query_madrid = [{"$geoNear" : {'near': [40.46879, -3.68915],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

In [41]:
geoloc_madrid = madrid.aggregate(query_madrid)

In [42]:
madrid_json = json.loads(dumps(geoloc_madrid))

In [43]:
madrid_df = pd.DataFrame(madrid_json)

In [44]:
# As we are not going to need the followinf columns for this purpose, we remove them
gf.drop_columns(madrid_df)

In [45]:
madrid_df["city"] = "Madrid"

In [46]:
madrid_df['shortname'] = madrid_df['shortname'].map(diccionario)

In [47]:
madrid_df.shortname.unique()

array([nan, 'Bar', 'Train Station', 'Starbucks'], dtype=object)

# DATA FOR PONDERATIONS

In [48]:
data = madrid_df.append([lisbon_df, london_df])

In [49]:
data["Puntuation"] = data["shortname"].apply(gf.puntuacion)

In [50]:
data["Total"] = data["distance"]*data["Puntuation"]

In [51]:
data_final = data.groupby(['city'])["Total"].sum()

In [52]:
data_final

city
Lisbon    1553.270196
London    1374.092475
Madrid     917.717745
Name: Total, dtype: float64